## Import Libraries

We import the necessary libraries for constructing the default estimator and performing Weight of Evidence (WoE) binning.

In [26]:
# Import libraries
import pandas as pd
import numpy as np
import scorecardpy as sc
from datetime import datetime
# For visualization
import matplotlib.pyplot as plt
import seaborn as sns

In [27]:
sns.set_style("whitegrid")
plt.style.use('ggplot')

## Load the Engineered Dataset

We load the cleaned and engineered dataset from `data/processed/engineered_data.csv` to proceed with constructing the default estimator and WoE binning.

In [28]:
# Load the engineered dataset
data_path = "../data/processed/engineered_data.csv"
df = pd.read_csv(data_path)

# Convert TransactionStartTime to datetime format and ensure it's timezone-naive
df['TransactionStartTime'] = pd.to_datetime(df['TransactionStartTime']).dt.tz_localize(None)

# Display the first few rows
df.head()

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProductId,Amount,Value,...,ProductCategory_movies,ProductCategory_other,ProductCategory_ticket,ProductCategory_transport,ProductCategory_tv,ProductCategory_utility_bill,ChannelId_ChannelId_2,ChannelId_ChannelId_3,ChannelId_ChannelId_5,ProviderId_encoded
0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,UGX,256,ProductId_10,0.092004,0.000101,...,False,False,False,False,False,False,False,True,False,5
1,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,UGX,256,ProductId_6,0.091910,0.000002,...,False,False,False,False,False,False,True,False,False,3
2,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,UGX,256,ProductId_1,0.091958,0.000050,...,False,False,False,False,False,False,False,True,False,5
3,TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,UGX,256,ProductId_21,0.093750,0.002206,...,False,False,False,False,False,True,False,True,False,0
4,TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,UGX,256,ProductId_6,0.091853,0.000065,...,False,False,False,False,False,False,True,False,False,3


## Define Proxy Variable Using RFMS Framework

We construct a proxy variable to classify users as high-risk or low-risk using the RFMS framework:
- **Recency**: Days since the last transaction.
- **Frequency**: Number of transactions per customer.
- **Monetary**: Total transaction amount per customer.
- **Status**: FraudResult as an indicator of risky behavior.

The RFMS scores are normalized and combined into a single score. Users with RFMS scores below the 20th percentile are classified as high-risk, while others are classified as low-risk.

In [34]:
# Calculate RFMS components
rfms_scores = df.groupby('CustomerId').agg(
    Recency=('TransactionStartTime', lambda x: (datetime.now() - x.max()).days),
    Frequency=('TransactionId', 'count'),
    Monetary=('Amount', 'sum'),
    Status=('FraudResult', 'max')  # Use FraudResult as an indicator of risky behavior
).reset_index()

# Normalize RFMS scores
rfms_scores[['Recency', 'Frequency', 'Monetary']] = rfms_scores[['Recency', 'Frequency', 'Monetary']].apply(
    lambda x: (x - x.min()) / (x.max() - x.min())
)

# Combine RFMS scores into a single score
rfms_scores['RFMS_Score'] = (
    rfms_scores['Recency'] * 0.25 +
    rfms_scores['Frequency'] * 0.25 +
    rfms_scores['Monetary'] * 0.4 +
    rfms_scores['Status'] * 0.1
)

# Define threshold for high-risk vs low-risk
threshold = rfms_scores['RFMS_Score'].quantile(0.2)  # Top 20% are considered high-risk
rfms_scores['Risk_Label'] = np.where(rfms_scores['RFMS_Score'] < threshold, 'High-Risk', 'Low-Risk')

# Display the RFMS scores DataFrame to verify
print("RFMS Scores DataFrame:")
print(rfms_scores.head())

RFMS Scores DataFrame:
        CustomerId   Recency  Frequency  Monetary  Status  RFMS_Score  \
0     CustomerId_1  0.933333   0.000000  0.000048       0    0.233352   
1    CustomerId_10  0.933333   0.000000  0.000048       0    0.233352   
2  CustomerId_1001  0.988889   0.000978  0.001059       0    0.247890   
3  CustomerId_1002  0.288889   0.002445  0.002560       0    0.073858   
4  CustomerId_1003  0.133333   0.001222  0.001310       0    0.034163   

  Risk_Label  
0   Low-Risk  
1   Low-Risk  
2   Low-Risk  
3   Low-Risk  
4   Low-Risk  


In [35]:
# Merge risk labels back to the main dataframe
df = df.merge(rfms_scores[['CustomerId', 'Risk_Label']], on='CustomerId', how='left')

# Check if the merge was successful
if 'Risk_Label' not in df.columns:
    raise KeyError("Merge failed: 'Risk_Label' column is missing in the merged DataFrame.")

# Handle missing values in Risk_Label (if any)
df['Risk_Label'].fillna('Low-Risk', inplace=True)  # Assign default value of 'Low-Risk' for missing entries

# Create a binary target variable (1 for High-Risk, 0 for Low-Risk)
df['Target'] = df['Risk_Label'].map({'High-Risk': 1, 'Low-Risk': 0})

# Display the updated dataframe
print("Updated DataFrame with Risk_Label and Target:")
print(df[['CustomerId', 'Risk_Label', 'Target']].head())

Updated DataFrame with Risk_Label and Target:
        CustomerId Risk_Label  Target
0  CustomerId_4406   Low-Risk       0
1  CustomerId_4406   Low-Risk       0
2  CustomerId_4683   Low-Risk       0
3   CustomerId_988   Low-Risk       0
4   CustomerId_988   Low-Risk       0


/tmp/ipykernel_8390/551724084.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Risk_Label'].fillna('Low-Risk', inplace=True)  # Assign default value of 'Low-Risk' for missing entries


In [36]:
print("Columns in rfms_scores:", rfms_scores.columns.tolist())
print("First few rows of rfms_scores:")
print(rfms_scores.head())

Columns in rfms_scores: ['CustomerId', 'Recency', 'Frequency', 'Monetary', 'Status', 'RFMS_Score', 'Risk_Label']
First few rows of rfms_scores:
        CustomerId   Recency  Frequency  Monetary  Status  RFMS_Score  \
0     CustomerId_1  0.933333   0.000000  0.000048       0    0.233352   
1    CustomerId_10  0.933333   0.000000  0.000048       0    0.233352   
2  CustomerId_1001  0.988889   0.000978  0.001059       0    0.247890   
3  CustomerId_1002  0.288889   0.002445  0.002560       0    0.073858   
4  CustomerId_1003  0.133333   0.001222  0.001310       0    0.034163   

  Risk_Label  
0   Low-Risk  
1   Low-Risk  
2   Low-Risk  
3   Low-Risk  
4   Low-Risk  


In [37]:
print("Columns in df after merge:", df.columns.tolist())
print("First few rows of df after merge:")
print(df[['CustomerId', 'Risk_Label']].head())

Columns in df after merge: ['TransactionId', 'BatchId', 'AccountId', 'SubscriptionId', 'CustomerId', 'CurrencyCode', 'CountryCode', 'ProductId', 'Amount', 'Value', 'TransactionStartTime', 'PricingStrategy', 'FraudResult', 'TotalTransactionAmount', 'AverageTransactionAmount', 'TransactionCount', 'StdDevTransactionAmount', 'TransactionHour', 'TransactionDay', 'TransactionMonth', 'TransactionYear', 'ProductCategory_data_bundles', 'ProductCategory_financial_services', 'ProductCategory_movies', 'ProductCategory_other', 'ProductCategory_ticket', 'ProductCategory_transport', 'ProductCategory_tv', 'ProductCategory_utility_bill', 'ChannelId_ChannelId_2', 'ChannelId_ChannelId_3', 'ChannelId_ChannelId_5', 'ProviderId_encoded', 'Risk_Label_x', 'Target', 'Risk_Label_y', 'Risk_Label']
First few rows of df after merge:
        CustomerId Risk_Label
0  CustomerId_4406   Low-Risk
1  CustomerId_4406   Low-Risk
2  CustomerId_4683   Low-Risk
3   CustomerId_988   Low-Risk
4   CustomerId_988   Low-Risk


In [38]:
if rfms_scores['CustomerId'].duplicated().any():
    print("Duplicate CustomerIds found in rfms_scores!")

## Perform Weight of Evidence (WoE) Binning

We perform Weight of Evidence (WoE) binning on selected features using the `scorecardpy` package. The WoE values capture the relationship between each category or bin and the target variable (Risk_Label).

For each feature:
- We calculate WoE bins and visualize them.
- We replace the original values with their corresponding WoE values.

In [40]:
# Print the list of columns in the DataFrame
print(df.columns.tolist())

['TransactionId', 'BatchId', 'AccountId', 'SubscriptionId', 'CustomerId', 'CurrencyCode', 'CountryCode', 'ProductId', 'Amount', 'Value', 'TransactionStartTime', 'PricingStrategy', 'FraudResult', 'TotalTransactionAmount', 'AverageTransactionAmount', 'TransactionCount', 'StdDevTransactionAmount', 'TransactionHour', 'TransactionDay', 'TransactionMonth', 'TransactionYear', 'ProductCategory_data_bundles', 'ProductCategory_financial_services', 'ProductCategory_movies', 'ProductCategory_other', 'ProductCategory_ticket', 'ProductCategory_transport', 'ProductCategory_tv', 'ProductCategory_utility_bill', 'ChannelId_ChannelId_2', 'ChannelId_ChannelId_3', 'ChannelId_ChannelId_5', 'ProviderId_encoded', 'Risk_Label_x', 'Target', 'Risk_Label_y', 'Risk_Label']


In [39]:
# Select features for WoE binning
features = ['ChannelId', 'ProductCategory', 'ProviderId', 'Amount', 'Value']

# Perform WoE binning
bins = sc.woebin(df[features + ['Target']], y='Target')

# Plot WoE bins for each feature
for feature, bin_table in bins.items():
    plt.figure(figsize=(8, 4))
    sc.woebin_plot(bin_table)
    plt.title(f"Weight of Evidence (WoE) Bins for {feature}")
    plt.show()

# Replace original values with WoE values
df = sc.woebin_ply(df, bins)

# Display the updated dataframe
df.head()

KeyError: "['ChannelId', 'ProductCategory', 'ProviderId'] not in index"

## Calculate Information Value (IV)

We calculate the Information Value (IV) for each feature to assess its predictive power. Features with higher IV values are better predictors of the target variable.

The IV values are visualized using a bar chart for clarity.

In [ ]:
# Extract IV values from WoE bins
iv_values = {feature: bins[feature]['total_iv'][0] for feature in features}

# Display IV results
iv_df = pd.DataFrame.from_dict(iv_values, orient='index', columns=['Information_Value'])
iv_df.sort_values(by='Information_Value', ascending=False, inplace=True)

# Visualize IV values
plt.figure(figsize=(8, 5))
sns.barplot(x=iv_df.index, y=iv_df['Information_Value'], palette='viridis')
plt.title("Information Value (IV) for Each Feature")
plt.xticks(rotation=45)
plt.show()

# Display IV table
iv_df

## Save the Updated Dataset

We save the updated dataset with WoE-transformed features to `data/processed/default_estimator_data.csv` for use in subsequent tasks such as modeling.

In [ ]:
# Save the updated dataset
updated_data_path = "../data/processed/default_estimator_data.csv"
df.to_csv(updated_data_path, index=False)

print(f"Updated dataset saved to {updated_data_path}")